In [12]:
import dataset
X_train, X_test, y_train, y_test, train_users, train_items, test_users, test_items = dataset.vectorize()

In [36]:
import numpy as np
import torch
from torch import optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class FM(nn.Module):
   def __init__(self, num_feature, num_v_feature):
      # num_feature : dataset에서 변수의 개수 
      # num_v_feature : v 벡터의 feature, hyperparameter
      super(FM, self).__init__()
      self._w0 = nn.Parameter(torch.zeros(1)) 
      self._wi = nn.Parameter(torch.randn((num_feature)))
      self._vif = nn.Parameter(torch.randn((num_feature, num_v_feature)))
      

   def forward(self,input):
      # input = torch.FloatTensor(input)
      linear_terms = self._wi.matmul(input)
      interactions = 0.5 * ( sum( pow(input.matmul(self._vif),2) - pow(input,2).matmul(pow(self._vif,2)) ) )

      y_hat = torch.sigmoid(self._w0 + linear_terms + interactions)

      return y_hat

In [40]:
class FactorizationMachine(nn.Module):
    
    def __init__(self, field_dims, latent_dims):
        super(FactorizationMachine, self).__init__()
        
        self.w_0 = nn.Parameter(nn.init.normal_(torch.zeros((1, ))), requires_grad=True) #(1, )
        self.w_i = nn.Parameter(nn.init.normal_(torch.zeros((1, field_dims)), std=1.0/field_dims), requires_grad = True) # (1, 4308)
        self.V = nn.Parameter(nn.init.normal_(torch.zeros((field_dims, latent_dims)), std=1.0/latent_dims), requires_grad = True) # (4308, 40)
        
        
    def forward(self, x):
        """
        batch shape (64, 4308)
        """
        temp_1 = self.w_0 + torch.matmul(x, self.w_i.T) # (64, 1)
        
        square_of_sum = torch.sum(torch.matmul(x, self.V), dim = 1) ** 2
        sum_of_square = torch.sum(torch.matmul(x, self.V) ** 2, dim = 1)
        temp_2 = (square_of_sum - sum_of_square).view(-1, 1)
        
        result = temp_1 + 0.5 * temp_2
        
        return result
    
    def init_weight(self):
        pass


In [41]:
class MovielensDataset(Dataset):
    """
    torch.utils.data.Dataset 상속
    """
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.size(0)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [81]:
def get_data_loader(batch_size):
   torch.manual_seed(0)
   train_dataset = MovielensDataset(X = torch.FloatTensor(X_train.toarray()),
                                y = y_train)
   test_dataset = MovielensDataset(X = torch.FloatTensor(X_test.toarray()),
                                y = y_test)
   train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
   test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
   return train_loader, test_loader

train_loader, test_loader = get_data_loader(batch_size = 64)


In [82]:
# check if cuda available
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(315)
if device == "cuda":
    torch.cuda.manual_seed_all(912)

device

'cpu'

In [96]:
import time

# check if cuda available
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(315)
if device == "cuda":
    torch.cuda.manual_seed_all(912)

_, num_feature= X_train.shape
num_v_feature = 15

model = FactorizationMachine(num_feature, num_v_feature)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


def train(model):
   model.train()
   total_loss = 0
   start = time.time()

   for i, (input, label) in enumerate(train_loader):
      y_pred = model(input)
      optimizer.zero_grad()
      loss = criterion(y_pred.flatten().float(), label.float())
      # compute gradient and do SGD step
      loss.backward()
      optimizer.step()
      total_loss += loss
   end = time.time() #epoch time
   print('==>  Train Loss: {loss:.3f}, 1 epoch time : {time:.3f}'.format(loss=total_loss,time=end-start))

def validation(model):
      model.eval()
      criterion = nn.MSELoss()
      for i, (input, label) in enumerate(test_loader):
         output = model(input)
         criterion_ = criterion(output.flatten().float(),label.float())
         RMSE = torch.sqrt(criterion_)
      
      print('==> Test RMSE {:.3f}'.format(RMSE))
      return RMSE

   
   

for epoch in range(10):
   train(model)
   validation(model)
      

==>  Train Loss: 1850.984, 1 epoch time : 2.947
==> Test RMSE 0.880
==>  Train Loss: 1657.602, 1 epoch time : 2.654
==> Test RMSE 1.039
==>  Train Loss: 1587.339, 1 epoch time : 2.661
==> Test RMSE 1.101
==>  Train Loss: 1535.333, 1 epoch time : 2.689
==> Test RMSE 1.266
==>  Train Loss: 1496.416, 1 epoch time : 2.675
==> Test RMSE 1.077
==>  Train Loss: 1466.062, 1 epoch time : 3.018
==> Test RMSE 0.866
==>  Train Loss: 1440.992, 1 epoch time : 2.668
==> Test RMSE 0.975
==>  Train Loss: 1419.641, 1 epoch time : 3.102
==> Test RMSE 0.883
==>  Train Loss: 1402.229, 1 epoch time : 2.727
==> Test RMSE 1.034
==>  Train Loss: 1385.636, 1 epoch time : 2.947
==> Test RMSE 0.994


In [76]:
loss

tensor([4., 2., 5., 1., 5., 3., 3., 3., 3., 4., 4., 1., 4., 5., 4., 5., 4., 2.,
        3., 5., 3., 4., 4., 1., 3., 4., 3., 4., 4., 3., 3., 5., 4., 3., 3., 4.,
        5., 2., 5., 5., 4., 4., 4., 4., 4., 4., 4., 3., 5., 5., 4., 4., 3., 4.,
        2., 3., 2., 3., 3., 2., 5., 4., 5., 1.], dtype=torch.float64)

In [90]:
loss.backward()

RuntimeError: Found dtype Double but expected Float

In [94]:
for i, (input, label) in enumerate(train_loader):
      y_pred = model(input)
      optimizer.zero_grad()
      loss = criterion(y_pred.flatten().float(), label.float())
      # compute gradient and do SGD step
      loss.backward()
      optimizer.step()
      total_loss += loss


NameError: name 'total_loss' is not defined

tensor(4.6394, dtype=torch.float64, grad_fn=<MseLossBackward>)

In [68]:
loss

tensor(6.1473, dtype=torch.float64, grad_fn=<MseLossBackward>)

In [52]:
loss.backward()

RuntimeError: Found dtype Double but expected Float

In [35]:
model(input)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x2623 and 64x2623)

In [39]:
for i, (input, label) in enumerate(train_loader):
      y_pred = model(input)
      loss = criterion(y_pred.view(-1), label.view(-1))
      # compute gradient and do SGD step
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      

RuntimeError: Found dtype Double but expected Float

In [195]:
model = FM(num_feature, num_v_feature)
optimizer = optim.SGD(model.parameters(), lr=0.01)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
criterion = nn.MSELoss()


In [197]:
model.to(device)
X_train = torch.FloatTensor(X_train.toarray()).to(device)
X_test = torch.FloatTensor(X_test.toarray()).to(device)
Y_train = torch.FloatTensor(y_train).to(device)
Y_test =torch.FloatTensor(y_test).to(device)

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:73] data. DefaultCPUAllocator: not enough memory: you tried to allocate 950260440 bytes. Buy new RAM!

In [171]:
X_train.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [152]:
a, b = MovielensDataset(X_train, y_train)

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [67]:
import time
from torch.utils.data import Dataset, DataLoader

def validation(model):
      model.eval()
      criterion = nn.MSELoss()
      for input, label in zip(X_test, Y_test):
         output = model(input)
         criterion_ = criterion(output,label.reshape(1))
         RMSE = torch.sqrt(criterion_)
      
      print('==> Test RMSE {:.3f}'.format(RMSE))
      return RMSE

for epoch in range(10):
   model.train()
   criterion = nn.MSELoss()
   total_loss = 0
   start = time.time()
   for input, label in zip(X_train, Y_train):
      y_pred = model(input)
      loss = criterion(y_pred, label.reshape(1))
      # compute gradient and do SGD step
      optimizer.zero_grad()
      loss.backward()
      total_loss += loss
      optimizer.step()
   end = time.time() #batch time

   print(f'Iteration : {epoch} -- Total loss : {total_loss:.4f}, Time : {end-start:.4f}')
   validation(model)
      

Iteration : 0 -- Total loss : 691749.0000, Time : 264.6771
==> Test RMSE 1.000


KeyboardInterrupt: 

In [ ]:

train_loader

In [122]:
for i, (input, label) in enumerate(train_loader):
    print(i, input, label)

AttributeError: 'Tensor' object has no attribute 'toarray'

SyntaxError: invalid syntax (<ipython-input-120-31d81f55dfc9>, line 17)